# Data acquisition, cleansing and preparation

Import necesarios

In [ ]:
import requests
import time
import shelve
import pandas as pd
import numpy as np

Comprobamos si hay un archivo con los gameIds o si tenemos que generarlo

In [2]:
try:
    r = shelve.open('gameids.db')
    print (r['key'])
    gameIds = list(r['key'])
    r.close()
except:
    gameIds = []

['EUW1_5807870283', 'EUW1_5807898506', 'EUW1_5807904750', 'EUW1_5807903116', 'EUW1_5807840780', 'EUW1_5807699227', 'EUW1_5807664897', 'EUW1_5807662262', 'EUW1_5807617737', 'EUW1_5807613510', 'EUW1_5806756866', 'EUW1_5806669444', 'EUW1_5806653730', 'EUW1_5806617775', 'EUW1_5806593444', 'EUW1_5806469184', 'EUW1_5806463101', 'EUW1_5806358331', 'EUW1_5806412623', 'EUW1_5806278735', 'EUW1_5808872462', 'EUW1_5808437268', 'EUW1_5808463562', 'EUW1_5805947050', 'EUW1_5803854800', 'EUW1_5803743399', 'EUW1_5802409003', 'EUW1_5800473133', 'EUW1_5799263621', 'EUW1_5779206003', 'EUW1_5779191762', 'EUW1_5776829761', 'EUW1_5776790586', 'EUW1_5776387753', 'EUW1_5776210044', 'EUW1_5776164781', 'EUW1_5771540861', 'EUW1_5770845468', 'EUW1_5770811530', 'EUW1_5770615758', 'EUW1_5806140879', 'EUW1_5806027001', 'EUW1_5805849136', 'EUW1_5804170573', 'EUW1_5803076431', 'EUW1_5802703021', 'EUW1_5802346069', 'EUW1_5800979187', 'EUW1_5800184140', 'EUW1_5800161659', 'EUW1_5799453047', 'EUW1_5798587547', 'EUW1_57985

In [3]:
try:
    df = pd.read_pickle("gameinfo.pkl")
    firstTime = False
except:
    firstTime = True

Riot API key para las requests

In [4]:
api_key = "RGAPI-a8585106-4f99-4026-aa58-278a3c468d06"

Request para sacar la lista de los 300 mejores jugadores de europa (EUW)

In [5]:
response = requests.get("https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key="+api_key+"").json()

In [6]:
response

{'tier': 'CHALLENGER',
 'leagueId': '04a0f903-37c7-316f-af90-5f8eed42011c',
 'queue': 'RANKED_SOLO_5x5',
 'name': "Nunu's Lancers",
 'entries': [{'summonerId': 'UqjVHvl5Vkl63A0p1hBkK2BggdOPK0T8fPRFIRNHP5me06me4M4MlLCuOg',
   'summonerName': 'FERRETTT',
   'leaguePoints': 1094,
   'rank': 'I',
   'wins': 224,
   'losses': 190,
   'veteran': True,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': True},
  {'summonerId': 'X4n5ABsnmIvn-qelfdp7391lrMjxeoCAdhBmudpwjuWQ8SGs',
   'summonerName': 'twtv NyroLoL',
   'leaguePoints': 940,
   'rank': 'I',
   'wins': 254,
   'losses': 224,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'v7QOTiqJl-_G8WzlMqf-s3pStJ-GIdgB-1mM0uj8woOlgaE',
   'summonerName': 'Lumerion',
   'leaguePoints': 883,
   'rank': 'I',
   'wins': 406,
   'losses': 369,
   'veteran': True,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'juwKp3PocrWnBGM31HfuHKDO-cA2NHj6f

In [7]:
print(len(response["entries"]))

300


Guardamos sus PUUID para poder acceder mas tarde a sus partidas 

In [8]:
sumPUID = []
error_cont = 0
for i in range(0,len(response["entries"])):
    sumNames = response["entries"][i]["summonerName"]
    try:
      responsePUID = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+sumNames+"?api_key="+api_key+"").json()
      sumPUID.append(responsePUID["puuid"])
      time.sleep(2)
    except:
        error_cont += 1

if error_cont != 0:
    print("Cuentas no accesibles: "+ str(error_cont))

In [9]:
sumPUID

['MKEHE-7IioUEEIdMMqnULcXo3H803emFgCO6yyBQyz-aVF2h79nhoAFPGnLAGQjdkpd1Fye_11JXYA',
 'F1YXBav4O84urJAIyaU9VVpouF33fdoM_usByxHkaZUYw2-oECF8SmamjsDS08DRh2xDkIpRd0Gveg',
 '-kA7O4RLkrS_hz5sahSYHCMGZl1XR0HX_w62VON09qn14NVCyMAPf8RF4zI3FnzjUgnmGlWt1zv4cA',
 'ZtbEL3eXmW0CMLACkK_jpy89QbeC_A5aa6Y8xxaLgjOmv7Dia8TInxGjY9zAzzqWjP0XbjFTEihIsg',
 '6PMj5EeU5ABSu70_6EkxGaqlUG2VrBA1NDPx5WWAv6qc6QjzrtJK-y-p6SNyHsWRTLhR80ffy8DaoA',
 'YWPywm8BI2AYy_XlxM5lkbJSgzM245yV7Ss0tcYJIyQ13MAMFKK8JKHQfV1tZwLEVxl8WFJC2D4dEA',
 'EyYqyiuBbK6o--M5nIVROVZCspdVApJ1XF33URmLHcE129kOvOHsEDoJ9yoXaYba221IbhUDed9Ohg',
 'P6SxOoBky1Zd0I8XWjkeIkALWGxkC3wWZ6pJp0Lz8XUwMAFgQ_kBd11ZiuSdDIaLgSYn2ktrtl9BuA',
 'YX1bU9gKmLkg1ZnAF8Uw1ZyF-7G3QimsjI-v9Eq21LZjcn4eg_pOY7SwNPDLPh7XmBAaCk6Z0JJ1xA',
 'q-cdimLl7TAgZVOk8dcY3rbKa6ZO-pw7rcdJtwzLP5vOwKpR-T9m4yIepk23bgoxymyla5j8g9qgHQ',
 'U41TEaHn8IS7G-a4uYDRScsbrPLWxVNsbSa-XqK_yKwueGulWVDhIDK0pv31uAIFcDucUJ4ygn78hA',
 'e4kbuIn6oCi9MOqt3xnwRwHldZhbW01SXb7Mtiy6D2flqcOy2e1qWBmFTzukXdgE01sdlGBz_ljrMg',
 'IT

In [10]:
len(sumPUID)

300

Obtenemos los ultimos 20 games de cada jugador

In [11]:
newgameIds = []
error_cont = 0
for puuid in sumPUID:
    try:
      response = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/"+puuid+"/ids?queue=420&start=0&count=20&api_key="+api_key+"").json()
      for i in range(0,len(response)):
            if response[i] not in gameIds and response[i] not in newgameIds:
                newgameIds.append(response[i])
                
      time.sleep(2)
    except:
        error_cont += 1
        
if error_cont != 0:
    print("Game Ids no accesibles: "+ str(error_cont))

In [12]:
len(newgameIds)

469

Los añadimos a la base de datos de gameIds para que no esten repetidos en un futuro

In [13]:
for gid in newgameIds:
    gameIds.append(gid)

In [14]:
s = shelve.open('gameids.db')
s.clear()
s['key'] = gameIds

In [15]:
print(gameIds)

['EUW1_5807870283', 'EUW1_5807898506', 'EUW1_5807904750', 'EUW1_5807903116', 'EUW1_5807840780', 'EUW1_5807699227', 'EUW1_5807664897', 'EUW1_5807662262', 'EUW1_5807617737', 'EUW1_5807613510', 'EUW1_5806756866', 'EUW1_5806669444', 'EUW1_5806653730', 'EUW1_5806617775', 'EUW1_5806593444', 'EUW1_5806469184', 'EUW1_5806463101', 'EUW1_5806358331', 'EUW1_5806412623', 'EUW1_5806278735', 'EUW1_5808872462', 'EUW1_5808437268', 'EUW1_5808463562', 'EUW1_5805947050', 'EUW1_5803854800', 'EUW1_5803743399', 'EUW1_5802409003', 'EUW1_5800473133', 'EUW1_5799263621', 'EUW1_5779206003', 'EUW1_5779191762', 'EUW1_5776829761', 'EUW1_5776790586', 'EUW1_5776387753', 'EUW1_5776210044', 'EUW1_5776164781', 'EUW1_5771540861', 'EUW1_5770845468', 'EUW1_5770811530', 'EUW1_5770615758', 'EUW1_5806140879', 'EUW1_5806027001', 'EUW1_5805849136', 'EUW1_5804170573', 'EUW1_5803076431', 'EUW1_5802703021', 'EUW1_5802346069', 'EUW1_5800979187', 'EUW1_5800184140', 'EUW1_5800161659', 'EUW1_5799453047', 'EUW1_5798587547', 'EUW1_57985

In [16]:
len(gameIds)

4616

Creamos un dataframe con cada partida, en el se incluiran el gameId, equipo ganador y 10 variables a lo largo de 15 minutos de partida.

In [17]:
gamesless15 = 0
stats = {
        "gameId": "",
        "killsDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "goldDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "xpDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "csDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "drakesDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "heraldsDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "towersDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "platesDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "totaldmgDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "wardsDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "BlueWin": False
    }

for i in range(0,len(newgameIds)):
    print(i)
    response = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+newgameIds[i]+"/timeline?api_key="+api_key+"").json()
    time.sleep(2)
    
    
    if len(response["info"]["frames"]) > 16:
        
        kills = 0;ekills = 0
        gold = 0;egold = 0
        xp = 0;exp = 0
        cs = 0;ecs = 0
        drakes = 0;edrakes = 0
        heralds = 0;eheralds = 0
        towers = 0;etowers = 0
        plates = 0;eplates= 0
        dmg = 0;edmg = 0
        wards = 0;ewards = 0
        
        #First 15min loop                                
        for j in range(1,16):
            
            #Events loop
            for k in range(0,len(response["info"]["frames"][j]["events"])):
                
                #KillsDiff
                if response["info"]["frames"][j]["events"][k]["type"] == "CHAMPION_KILL":
                    if response["info"]["frames"][j]["events"][k]["killerId"] <= 5:
                        kills += 1
                    else:
                        ekills += 1 
                        
                #DrakesDiff & HeraldsDiff
                if response["info"]["frames"][j]["events"][k]["type"] == "ELITE_MONSTER_KILL":
                    if response["info"]["frames"][j]["events"][k]["monsterType"] == "DRAGON": 
                        if response["info"]["frames"][j]["events"][k]["killerId"] <= 5:
                            drakes += 1
                        else:
                            edrakes += 1 
                            
                    elif response["info"]["frames"][j]["events"][k]["monsterType"] == "RIFTHERALD":
                        if response["info"]["frames"][j]["events"][k]["killerId"] <= 5:
                            heralds += 1
                        else:
                            eheralds += 1
                
                #TowersDiff
                if response["info"]["frames"][j]["events"][k]["type"] == "BUILDING_KILL":
                    if response["info"]["frames"][j]["events"][k]["buildingType"] == "TOWER_BUILDING":
                        if response["info"]["frames"][j]["events"][k]["teamId"] == 200:
                            towers += 1
                        else:
                            etowers += 1
                            
                #PlatesDiff
                if response["info"]["frames"][j]["events"][k]["type"] == "TURRET_PLATE_DESTROYED":
                    if response["info"]["frames"][j]["events"][k]["teamId"] == 200:
                        plates += 1
                    else:
                        eplates += 1
                
                #WardsDiff
                if response["info"]["frames"][j]["events"][k]["type"] == "WARD_PLACED":
                    if response["info"]["frames"][j]["events"][k]["creatorId"] <= 5:
                        wards += 1
                    else:
                        ewards += 1
                            
                            
                            
            #Players Loop
            for k in range(1,11):
                
                #GoldDiff
                if k <= 5:
                    gold += response["info"]["frames"][j]["participantFrames"][str(k)]["totalGold"]
                else:
                    egold += response["info"]["frames"][j]["participantFrames"][str(k)]["totalGold"]
                    
                #XpDiff
                if k <= 5:
                    xp += response["info"]["frames"][j]["participantFrames"][str(k)]["xp"]
                else:
                    exp += response["info"]["frames"][j]["participantFrames"][str(k)]["xp"]  
                    
                #CsDiff
                if k <= 5:
                    cs += response["info"]["frames"][j]["participantFrames"][str(k)]["minionsKilled"] + response["info"]["frames"][j]["participantFrames"][str(k)]["jungleMinionsKilled"]
                else:
                    ecs += response["info"]["frames"][j]["participantFrames"][str(k)]["minionsKilled"] + response["info"]["frames"][j]["participantFrames"][str(k)]["jungleMinionsKilled"]
                
                #DmgDiff
                if k <= 5:
                    dmg += response["info"]["frames"][j]["participantFrames"][str(k)]["damageStats"]["totalDamageDoneToChampions"]
                else:
                    edmg += response["info"]["frames"][j]["participantFrames"][str(k)]["damageStats"]["totalDamageDoneToChampions"]
            
            
            
            #Final append to dict
            stats["killsDiff"][j-1] = kills - ekills
            stats["goldDiff"][j-1] = gold - egold
            stats["xpDiff"][j-1] = xp - exp
            stats["csDiff"][j-1] = cs - ecs
            stats["drakesDiff"][j-1] = drakes - edrakes
            stats["heraldsDiff"][j-1] = heralds - eheralds
            stats["towersDiff"][j-1] = towers - etowers
            stats["platesDiff"][j-1] = plates - eplates
            stats["totaldmgDiff"][j-1] = dmg - edmg
            stats["wardsDiff"][j-1] = wards - ewards

        for k in range(0,len(response["info"]["frames"][len(response["info"]["frames"])-1]["events"])):
            if response["info"]["frames"][len(response["info"]["frames"])-1]["events"][k]["type"] == "GAME_END":
                if response["info"]["frames"][len(response["info"]["frames"])-1]["events"][k]["winningTeam"] == 100:
                    stats["BlueWin"] = True
        
        stats["gameId"] = newgameIds[i]
        
        #Insert en pandas dataframe
        if firstTime == True:
            firstTime = False
            df = pd.DataFrame(columns=["gameId","killsDiff","goldDiff","xpDiff","csDiff","drakesDiff","heraldsDiff","towersDiff","platesDiff","totaldmgDiff","wardsDiff","BlueWin"])
            df = df.append(stats,ignore_index = True)
        else:
            df = df.append(stats,ignore_index = True)
            stats = {
                "gameId": "",
                "killsDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "goldDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "xpDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "csDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "drakesDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "heraldsDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "towersDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "platesDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "totaldmgDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "wardsDiff": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                "BlueWin": False
            }
    
    else:
        gamesless15+=1

if gamesless15 != 0:
    print("Games con menor duracion de 15 min: "+ str(gamesless15))

0


C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/2343622648.py:141: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(stats,ignore_index = True)


1


C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/2343622648.py:141: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(stats,ignore_index = True)


2


...

C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/2343622648.py:141: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(stats,ignore_index = True)


467


C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/2343622648.py:141: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(stats,ignore_index = True)


468
Games con menor duracion de 15 min: 5


C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/2343622648.py:141: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(stats,ignore_index = True)


In [19]:
df.head(3)

,gameId,killsDiff,goldDiff,xpDiff,csDiff,drakesDiff,heraldsDiff,towersDiff,platesDiff,totaldmgDiff,wardsDiff,BlueWin
1,EUW1_5807898506,"[0, 0, 0, -1, -2, -3, -5, -4, -4, -4, -7, -9, ...","[0, 32, -15, -317, -1352, -2959, -5704, -8324,...","[0, -15, -50, -76, -1849, -3272, -5799, -8208,...","[0, 0, -7, -9, -32, -48, -81, -119, -173, -255...","[0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1,...","[0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -2, -3, -3, -...","[0, 0, 0, 0, 0, -3, -4, -6, -10, -10, -14, -15...","[0, -44, -302, -1176, -2230, -4278, -7527, -94...","[0, 0, 0, 1, 3, 2, 3, 3, 4, 1, 0, 0, -3, -4, -6]",True
2,EUW1_5807904750,"[0, 0, 0, 0, -2, -2, -4, -3, -3, -5, -4, -5, -...","[0, 31, -115, 95, -968, -2078, -4581, -7012, -...","[0, 112, -256, -338, -964, -1365, -2913, -3933...","[0, 3, 1, 11, 17, 23, 6, -14, -43, -79, -119, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1]","[0, 0, 0, 0, 0, 0, 0, -2, -2, -7, -5, -5, -5, ...","[0, -60, -250, -713, -1176, -1251, -2839, -347...","[1, 1, 3, 2, 4, 5, 5, 6, 4, 2, 5, 3, 4, 7, 2]",False
3,EUW1_5807903116,"[0, 0, -1, 1, 1, 2, 2, 1, 3, 3, 4, 4, 5, 6, 9]","[0, 95, -413, 131, 786, 1406, 2229, 2856, 4454...","[40, 236, -125, -667, -1702, -2807, -2795, -31...","[0, 5, 12, 18, 18, 8, 9, 13, 15, 0, -10, -12, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3]","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 3, 6, 5, 6, 6]","[0, 72, -569, -436, -36, 684, 1002, 931, 2586,...","[5, 7, 11, 14, 19, 22, 23, 34, 41, 43, 51, 53,...",False


In [20]:
len(df)

3964

 Separamos los arrays en features independientes para poder analizarlas posteriormente

In [21]:
cnames = ["gd1","gd2","gd3","gd4","gd5","gd6","gd7","gd8","gd9","gd10","gd11","gd12","gd13","gd14","gd15"]
new = []
for i in range(0,len(df)):
    new.append(cnames)
df["temp_names"] = new

In [22]:
dfte = df.explode(['goldDiff', 'temp_names'])
dffinal = dfte.pivot(columns='temp_names', values='goldDiff', index='gameId')

In [23]:
bluewin = []
xpd1 = [];xpd2 = [];xpd3 = [];xpd4 = [];xpd5 = [];xpd6 = [];xpd7 = [];xpd8 = [];xpd9 = [];xpd10 = []
xpd11 = [];xpd12 = [];xpd13 = [];xpd14 = [];xpd15 = []
kd1 = [];kd2 = [];kd3 = [];kd4 = [];kd5 = [];kd6 = [];kd7 = [];kd8 = [];kd9 = [];kd10 = []
kd11 = [];kd12 = [];kd13 = [];kd14 = [];kd15 = []
csd1 = [];csd2 = [];csd3 = [];csd4 = [];csd5 = [];csd6 = [];csd7 = [];csd8 = [];csd9 = [];csd10 = []
csd11 = [];csd12 = [];csd13 = [];csd14 = [];csd15 = []
dd1 = [];dd2 = [];dd3 = [];dd4 = [];dd5 = [];dd6 = [];dd7 = [];dd8 = [];dd9 = [];dd10 = []
dd11 = [];dd12 = [];dd13 = [];dd14 = [];dd15 = []
hd1 = [];hd2 = [];hd3 = [];hd4 = [];hd5 = [];hd6 = [];hd7 = [];hd8 = [];hd9 = [];hd10 = []
hd11 = [];hd12 = [];hd13 = [];hd14 = [];hd15 = []
td1 = [];td2 = [];td3 = [];td4 = [];td5 = [];td6 = [];td7 = [];td8 = [];td9 = [];td10 = []
td11 = [];td12 = [];td13 = [];td14 = [];td15 = []
pd1 = [];pd2 = [];pd3 = [];pd4 = [];pd5 = [];pd6 = [];pd7 = [];pd8 = [];pd9 = [];pd10 = []
pd11 = [];pd12 = [];pd13 = [];pd14 = [];pd15 = []
tdd1 = [];tdd2 = [];tdd3 = [];tdd4 = [];tdd5 = [];tdd6 = [];tdd7 = [];tdd8 = [];tdd9 = [];tdd10 = []
tdd11 = [];tdd12 = [];tdd13 = [];tdd14 = [];tdd15 = []
wd1 = [];wd2 = [];wd3 = [];wd4 = [];wd5 = [];wd6 = [];wd7 = [];wd8 = [];wd9 = [];wd10 = []
wd11 = [];wd12 = [];wd13 = [];wd14 = [];wd15 = []

for i in range(0,len(dffinal)):
    #BlueWin
    bluewin.append(df[df["gameId"] == dffinal.iloc[i].name]["BlueWin"].item())
    
    #xpDiff
    xpd1.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[0])
    xpd2.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[1])
    xpd3.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[2])
    xpd4.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[3])
    xpd5.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[4])
    xpd6.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[5])
    xpd7.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[6])
    xpd8.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[7])
    xpd9.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[8])
    xpd10.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[9])
    xpd11.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[10])
    xpd12.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[11])
    xpd13.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[12])
    xpd14.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[13])
    xpd15.append(df[df["gameId"] == dffinal.iloc[i].name]["xpDiff"].item()[14])
    
    #killsDiff
    kd1.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[0])
    kd2.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[1])
    kd3.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[2])
    kd4.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[3])
    kd5.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[4])
    kd6.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[5])
    kd7.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[6])
    kd8.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[7])
    kd9.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[8])
    kd10.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[9])
    kd11.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[10])
    kd12.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[11])
    kd13.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[12])
    kd14.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[13])
    kd15.append(df[df["gameId"] == dffinal.iloc[i].name]["killsDiff"].item()[14])
    
    #killsDiff
    csd1.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[0])
    csd2.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[1])
    csd3.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[2])
    csd4.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[3])
    csd5.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[4])
    csd6.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[5])
    csd7.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[6])
    csd8.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[7])
    csd9.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[8])
    csd10.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[9])
    csd11.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[10])
    csd12.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[11])
    csd13.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[12])
    csd14.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[13])
    csd15.append(df[df["gameId"] == dffinal.iloc[i].name]["csDiff"].item()[14])
    
    #drakesDiff
    dd1.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[0])
    dd2.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[1])
    dd3.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[2])
    dd4.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[3])
    dd5.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[4])
    dd6.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[5])
    dd7.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[6])
    dd8.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[7])
    dd9.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[8])
    dd10.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[9])
    dd11.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[10])
    dd12.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[11])
    dd13.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[12])
    dd14.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[13])
    dd15.append(df[df["gameId"] == dffinal.iloc[i].name]["drakesDiff"].item()[14])
    
    #heraldsDiff
    hd1.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[0])
    hd2.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[1])
    hd3.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[2])
    hd4.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[3])
    hd5.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[4])
    hd6.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[5])
    hd7.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[6])
    hd8.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[7])
    hd9.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[8])
    hd10.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[9])
    hd11.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[10])
    hd12.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[11])
    hd13.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[12])
    hd14.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[13])
    hd15.append(df[df["gameId"] == dffinal.iloc[i].name]["heraldsDiff"].item()[14])
    
    #towersDiff
    td1.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[0])
    td2.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[1])
    td3.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[2])
    td4.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[3])
    td5.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[4])
    td6.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[5])
    td7.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[6])
    td8.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[7])
    td9.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[8])
    td10.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[9])
    td11.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[10])
    td12.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[11])
    td13.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[12])
    td14.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[13])
    td15.append(df[df["gameId"] == dffinal.iloc[i].name]["towersDiff"].item()[14])
    
    #platesDiff
    pd1.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[0])
    pd2.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[1])
    pd3.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[2])
    pd4.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[3])
    pd5.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[4])
    pd6.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[5])
    pd7.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[6])
    pd8.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[7])
    pd9.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[8])
    pd10.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[9])
    pd11.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[10])
    pd12.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[11])
    pd13.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[12])
    pd14.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[13])
    pd15.append(df[df["gameId"] == dffinal.iloc[i].name]["platesDiff"].item()[14])
    
    #totaldmgDiff
    tdd1.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[0])
    tdd2.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[1])
    tdd3.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[2])
    tdd4.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[3])
    tdd5.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[4])
    tdd6.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[5])
    tdd7.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[6])
    tdd8.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[7])
    tdd9.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[8])
    tdd10.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[9])
    tdd11.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[10])
    tdd12.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[11])
    tdd13.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[12])
    tdd14.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[13])
    tdd15.append(df[df["gameId"] == dffinal.iloc[i].name]["totaldmgDiff"].item()[14])
    
    #wardsDiff
    wd1.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[0])
    wd2.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[1])
    wd3.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[2])
    wd4.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[3])
    wd5.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[4])
    wd6.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[5])
    wd7.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[6])
    wd8.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[7])
    wd9.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[8])
    wd10.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[9])
    wd11.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[10])
    wd12.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[11])
    wd13.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[12])
    wd14.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[13])
    wd15.append(df[df["gameId"] == dffinal.iloc[i].name]["wardsDiff"].item()[14])

#----
dffinal["kd1"] = kd1
dffinal["kd2"] = kd2
dffinal["kd3"] = kd3
dffinal["kd4"] = kd4
dffinal["kd5"] = kd5
dffinal["kd6"] = kd6
dffinal["kd7"] = kd7
dffinal["kd8"] = kd8
dffinal["kd9"] = kd9
dffinal["kd10"] = kd10
dffinal["kd11"] = kd11
dffinal["kd12"] = kd12
dffinal["kd13"] = kd13
dffinal["kd14"] = kd14
dffinal["kd15"] = kd15
#----
dffinal["xpd1"] = xpd1
dffinal["xpd2"] = xpd2
dffinal["xpd3"] = xpd3
dffinal["xpd4"] = xpd4
dffinal["xpd5"] = xpd5
dffinal["xpd6"] = xpd6
dffinal["xpd7"] = xpd7
dffinal["xpd8"] = xpd8
dffinal["xpd9"] = xpd9
dffinal["xpd10"] = xpd10
dffinal["xpd11"] = xpd11
dffinal["xpd12"] = xpd12
dffinal["xpd13"] = xpd13
dffinal["xpd14"] = xpd14
dffinal["xpd15"] = xpd15
#-----
dffinal["csd1"] = csd1
dffinal["csd2"] = csd2
dffinal["csd3"] = csd3
dffinal["csd4"] = csd4
dffinal["csd5"] = csd5
dffinal["csd6"] = csd6
dffinal["csd7"] = csd7
dffinal["csd8"] = csd8
dffinal["csd9"] = csd9
dffinal["csd10"] = csd10
dffinal["csd11"] = csd11
dffinal["csd12"] = csd12
dffinal["csd13"] = csd13
dffinal["csd14"] = csd14
dffinal["csd15"] = csd15
#-----
dffinal["dd1"] = dd1
dffinal["dd2"] = dd2
dffinal["dd3"] = dd3
dffinal["dd4"] = dd4
dffinal["dd5"] = dd5
dffinal["dd6"] = dd6
dffinal["dd7"] = dd7
dffinal["dd8"] = dd8
dffinal["dd9"] = dd9
dffinal["dd10"] = dd10
dffinal["dd11"] = dd11
dffinal["dd12"] = dd12
dffinal["dd13"] = dd13
dffinal["dd14"] = dd14
dffinal["dd15"] = dd15
#-----
dffinal["hd1"] = hd1
dffinal["hd2"] = hd2
dffinal["hd3"] = hd3
dffinal["hd4"] = hd4
dffinal["hd5"] = hd5
dffinal["hd6"] = hd6
dffinal["hd7"] = hd7
dffinal["hd8"] = hd8
dffinal["hd9"] = hd9
dffinal["hd10"] = hd10
dffinal["hd11"] = hd11
dffinal["hd12"] = hd12
dffinal["hd13"] = hd13
dffinal["hd14"] = hd14
dffinal["hd15"] = hd15
#-----
dffinal["td1"] = td1
dffinal["td2"] = td2
dffinal["td3"] = td3
dffinal["td4"] = td4
dffinal["td5"] = td5
dffinal["td6"] = td6
dffinal["td7"] = td7
dffinal["td8"] = td8
dffinal["td9"] = td9
dffinal["td10"] = td10
dffinal["td11"] = td11
dffinal["td12"] = td12
dffinal["td13"] = td13
dffinal["td14"] = td14
dffinal["td15"] = td15
#-----
dffinal["pd1"] = pd1
dffinal["pd2"] = pd2
dffinal["pd3"] = pd3
dffinal["pd4"] = pd4
dffinal["pd5"] = pd5
dffinal["pd6"] = pd6
dffinal["pd7"] = pd7
dffinal["pd8"] = pd8
dffinal["pd9"] = pd9
dffinal["pd10"] = pd10
dffinal["pd11"] = pd11
dffinal["pd12"] = pd12
dffinal["pd13"] = pd13
dffinal["pd14"] = pd14
dffinal["pd15"] = pd15
#-----
dffinal["tdd1"] = tdd1
dffinal["tdd2"] = tdd2
dffinal["tdd3"] = tdd3
dffinal["tdd4"] = tdd4
dffinal["tdd5"] = tdd5
dffinal["tdd6"] = tdd6
dffinal["tdd7"] = tdd7
dffinal["tdd8"] = tdd8
dffinal["tdd9"] = tdd9
dffinal["tdd10"] = tdd10
dffinal["tdd11"] = tdd11
dffinal["tdd12"] = tdd12
dffinal["tdd13"] = tdd13
dffinal["tdd14"] = tdd14
dffinal["tdd15"] = tdd15
#-----
dffinal["wd1"] = wd1
dffinal["wd2"] = wd2
dffinal["wd3"] = wd3
dffinal["wd4"] = wd4
dffinal["wd5"] = wd5
dffinal["wd6"] = wd6
dffinal["wd7"] = wd7
dffinal["wd8"] = wd8
dffinal["wd9"] = wd9
dffinal["wd10"] = wd10
dffinal["wd11"] = wd11
dffinal["wd12"] = wd12
dffinal["wd13"] = wd13
dffinal["wd14"] = wd14
dffinal["wd15"] = wd15
#----
dffinal["BlueWin"] = bluewin

C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/3757335724.py:284: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dffinal["pd10"] = pd10
C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/3757335724.py:285: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dffinal["pd11"] = pd11
C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/3757335724.py:286: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/3757335724.py:310: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dffinal["wd4"] = wd4
C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/3757335724.py:311: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dffinal["wd5"] = wd5
C:\Users\Mario\AppData\Local\Temp/ipykernel_11176/3757335724.py:312: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

In [24]:
dffinal

temp_names,gd1,gd10,gd11,gd12,gd13,gd14,gd15,gd2,gd3,gd4,...,wd7,wd8,wd9,wd10,wd11,wd12,wd13,wd14,wd15,BlueWin
gameId,,,,,,,,,,,,,,,,,,,,,
EUW1_5763325310,516,16400,23181,30858,39067,49680,61268,1109,1774,3458,...,5,11,12,16,15,24,28,30,36,True
EUW1_5763410685,500,5005,5207,5813,6097,6703,7860,974,1228,1073,...,65,66,70,73,86,91,119,136,145,True
EUW1_5763819470,559,-2886,-4284,-6152,-6830,-8325,-8928,1097,1600,1046,...,19,24,28,30,37,41,48,50,54,True
EUW1_5764707648,0,-5689,-6628,-9264,-12651,-17040,-21966,162,-368,-1060,...,-7,-7,-7,-7,-9,-10,-9,-13,-13,False
EUW1_5765338010,0,8681,10020,11340,12929,14682,16479,-57,416,1796,...,3,7,4,4,1,2,1,0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EUW1_5819333656,0,3812,5223,5043,4635,3994,3570,108,77,285,...,5,4,5,2,2,2,2,-1,-2,False
EUW1_5819334145,0,2941,4293,4905,3935,3203,2481,136,580,52,...,-1,0,2,1,0,0,-4,-1,-3,True
EUW1_5819336784,0,16708,20028,23739,25954,28593,30409,83,236,1588,...,23,25,32,37,38,40,47,52,60,True


Comprobamos que la información del df se ha guardado correctamente en la bdd de pickle

In [39]:
df = pd.read_pickle("gameinfo.pkl")

In [26]:
dffinal.to_pickle("finalinfo.pkl")

In [61]:
dfprueban = pd.read_pickle("gameinfo.pkl")

In [62]:
dfprueban

,gameId,killsDiff,goldDiff,xpDiff,csDiff,drakesDiff,heraldsDiff,towersDiff,platesDiff,totaldmgDiff,wardsDiff,BlueWin
0,EUW1_5807870283,"[0, 0, 0, -1, -2, -3, -5, -4, -4, -4, -7, -9, ...","[0, 32, -15, -317, -1352, -2959, -5704, -8324,...","[0, -15, -50, -76, -1849, -3272, -5799, -8208,...","[0, 0, -7, -9, -32, -48, -81, -119, -173, -255...","[0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1,...","[0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -2, -3, -3, -...","[0, 0, 0, 0, 0, -3, -4, -6, -10, -10, -14, -15...","[0, -44, -302, -1176, -2230, -4278, -7527, -94...","[0, 0, 0, 1, 3, 2, 3, 3, 4, 1, 0, 0, -3, -4, -6]",True
1,EUW1_5807898506,"[0, 0, 0, -1, -2, -3, -5, -4, -4, -4, -7, -9, ...","[0, 32, -15, -317, -1352, -2959, -5704, -8324,...","[0, -15, -50, -76, -1849, -3272, -5799, -8208,...","[0, 0, -7, -9, -32, -48, -81, -119, -173, -255...","[0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1,...","[0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -2, -3, -3, -...","[0, 0, 0, 0, 0, -3, -4, -6, -10, -10, -14, -15...","[0, -44, -302, -1176, -2230, -4278, -7527, -94...","[0, 0, 0, 1, 3, 2, 3, 3, 4, 1, 0, 0, -3, -4, -6]",True
2,EUW1_5807904750,"[0, 0, 0, 0, -2, -2, -4, -3, -3, -5, -4, -5, -...","[0, 31, -115, 95, -968, -2078, -4581, -7012, -...","[0, 112, -256, -338, -964, -1365, -2913, -3933...","[0, 3, 1, 11, 17, 23, 6, -14, -43, -79, -119, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1]","[0, 0, 0, 0, 0, 0, 0, -2, -2, -7, -5, -5, -5, ...","[0, -60, -250, -713, -1176, -1251, -2839, -347...","[1, 1, 3, 2, 4, 5, 5, 6, 4, 2, 5, 3, 4, 7, 2]",False
3,EUW1_5807903116,"[0, 0, -1, 1, 1, 2, 2, 1, 3, 3, 4, 4, 5, 6, 9]","[0, 95, -413, 131, 786, 1406, 2229, 2856, 4454...","[40, 236, -125, -667, -1702, -2807, -2795, -31...","[0, 5, 12, 18, 18, 8, 9, 13, 15, 0, -10, -12, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3]","[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 3, 6, 5, 6, 6]","[0, 72, -569, -436, -36, 684, 1002, 931, 2586,...","[5, 7, 11, 14, 19, 22, 23, 34, 41, 43, 51, 53,...",False
4,EUW1_5807840780,"[0, 0, 0, -1, -1, -3, -3, -4, -7, -8, -7, -8, ...","[0, 26, 59, -466, -447, -2108, -3856, -6134, -...","[0, -68, -181, -1007, -661, -2146, -3450, -532...","[0, 0, 0, -7, 16, 24, 36, 44, 51, 60, 80, 105,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1]","[0, 0, 0, 0, 0, 0, -1, -1, -3, -4, -5, -6, -6,...","[0, 148, 406, 140, 225, -752, -2005, -4777, -8...","[0, 0, 0, 1, 2, 1, 0, 0, -2, -2, -3, -4, 0, 0,...",False
...,...,...,...,...,...,...,...,...,...,...,...,...
3496,EUW1_5809368228,"[0, 0, 0, 0, 0, 2, 2, 1, 0, -2, 1, 1, -1, 0, 1]","[-60, -147, -119, -251, -340, 420, 970, 904, -...","[0, -132, 18, -99, 328, 1318, 2228, 2286, 1848...","[0, 1, 11, 15, 24, 37, 48, 48, 48, 41, 39, 35,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, -1, -1, -3, -4, -3, -3, -4,...","[0, 113, 775, 1730, 2470, 4197, 5054, 5753, 57...","[-1, -1, -1, 1, 2, 2, 4, 2, 0, -5, -8, -13, -1...",False
3497,EUW1_5810209138,"[0, -1, -2, 0, 0, -1, 0, 0, 1, 0, -2, 0, 0, 0, 0]","[40, -280, -779, -246, 446, 1225, 2395, 4200, ...","[0, 236, 297, 485, 659, 1379, 2506, 4581, 7175...","[0, 0, -3, -2, 4, 32, 51, 91, 138, 183, 231, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3]","[0, 0, 0, 0, 0, 1, 1, 2, 4, 4, 5, 7, 7, 11, 11]","[-161, 411, 1361, 4005, 7498, 12185, 17825, 24...","[-1, -1, -1, -1, 0, 0, 1, -1, 1, 0, 3, 1, 3, 5...",True
3498,EUW1_5812517238,"[0, 0, 0, -1, -3, -4, -6, -7, -5, -5, -6, -6, ...","